### CNN
* http://andrew.gibiansky.com/blog/machine-learning/convolutional-neural-networks/
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721
* https://algotravelling.com/ru/%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5-%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5-%D1%8D%D1%82%D0%BE-%D0%B2%D0%B5%D1%81%D0%B5%D0%BB%D0%BE-3/
* https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
* https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b
* https://www.asozykin.ru/courses/nnpython
* https://www.tensorflow.org/tutorials/deep_cnn#convolutional-neural-networks

In [10]:
from scipy import ndimage
import numpy as np
import pandas as pd
import os
from IPython.display import display, Image
from scipy import misc
import matplotlib.pyplot as plt
import tensorflow
import seaborn

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

### Keras

In [15]:
img_width = 32
img_height = 32

# faced with the issue https://stackoverflow.com/a/39851572/5151861
#                      https://github.com/fchollet/keras/issues/3945#issuecomment-274321680

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model.add(Conv2D(32, (3, 3), data_format="channels_last", padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model.add(Conv2D(64, (3, 3), data_format="channels_last", padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

In [ ]:
model = Sequential()
model.add(Conv2D(16, (5, 5), strides=(2, 2), padding='same', kernel_initializer='he_uniform', 
           use_bias=False, input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization(scale=False))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization(scale=False))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (1, 1), padding='same', kernel_initializer='he_uniform', use_bias=False))
model.add(BatchNormalization(scale=False))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(n_classes, kernel_initializer='he_uniform'))
model.add(Activation('softmax'))

In [16]:
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [25]:
batch_size = 16
n_classes = 257

dir_train = 'data/keras/train'
dir_val = 'data/keras/validation'
dir_test = 'data/keras/test'

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    dir_train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = val_datagen.flow_from_directory(
    dir_val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = val_datagen.flow_from_directory(
    dir_test,
    target_size=(img_width, img_height),
    class_mode=None,
    batch_size=batch_size,
    shuffle=False)

Found 17263 images belonging to 257 classes.
Found 5634 images belonging to 257 classes.
Found 7710 images belonging to 1 classes.


In [27]:
model.fit_generator(
    train_generator,
    steps_per_epoch=200,
    epochs=20,
    validation_data=test_generator,
    validation_steps=200)

#model.save('first_try.h5')

Epoch 1/20
199/200 [============================>.] - ETA: 0s - loss: 5.2685 - acc: 0.0537

ValueError: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: [[[[ 0.44313729  0.59215689  0.68235296]
   [ 0.52549022  0.60000002  0.57647061]
   [ 0.58431375  0.58039218  0.47058827]
   ..., 
   [ 0.54509807  0.54509807  0.57647061]
   [ 0.50588238  0.53333336  0.50196081]
   [ 0.46274513  0.5411765   0.44313729]]

  [[ 0.52941179  0.59215689  0.65098041]
   [ 0.60000002  0.60784316  0.55686277]
   [ 0.65098041  0.59215689  0.47058827]
   ..., 
   [ 0.58431375  0.56078434  0.60000002]
   [ 0.54509807  0.54901963  0.52549022]
   [ 0.50196081  0.55686277  0.4666667 ]]

  [[ 0.63529414  0.60000002  0.61960787]
   [ 0.69411767  0.61960787  0.5529412 ]
   [ 0.71764708  0.6156863   0.4784314 ]
   ..., 
   [ 0.63137257  0.58431375  0.63137257]
   [ 0.60392159  0.56470591  0.55686277]
   [ 0.56470591  0.56862748  0.49803925]]

  ..., 
  [[ 0.80784321  0.60000002  0.71764708]
   [ 0.71372551  0.71764708  0.69803923]
   [ 0.64705884  0.80392164  0.7019608 ]
   ..., 

In [ ]:
results = model.predict_generator(test_generator, steps=nb_test_samples//64+1)
results = np.argmax(results, axis=1)+1

# create submission dataframe (in specified kaggle format)
sub = pd.DataFrame(
    data=[row for row in zip([i.split("/")[1] for i in test_generator.filenames], results.astype(int).tolist())],
    columns=['image', 'class']
)

In [ ]:
sub.to_csv('keras.csv', encoding='utf-8', index=False)

In [ ]:
from keras.models import model_from_json
import h5py
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")